In [ ]:
# Libraries

In [ ]:
import numpy as np

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
import cv2 as cv

In [ ]:
from sklearn import linear_model

In [ ]:
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
plt.style.use('seaborn')

In [ ]:
# Helping Functions

In [ ]:
def plot_image(image):
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    plt.show(block=True)

In [ ]:
def display_image(image):
    cv.imshow('image', image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [ ]:
def MinMaxScalar(x, a=-1, b=1):
    N = (x - x.min()) * (b - a)
    D = (x.max() - x.min())
    return (N / D) + a

In [ ]:
# Activate Functions

In [ ]:
def sigmoid(t):
    return 1 / (1 + np.exp(-t))